In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import ipywidgets
import matplotlib.pyplot as plt
# plt.rcParams['text.usetex'] = True
import pipeline
import deltas
import plots
import radius

# play with deltas on data

In [2]:
N1 = 100
N2 = 100
data_clf = pipeline.get_data_and_classifier(N1=N1, N2=N2, _plot=False)
data_info = pipeline.data_project_and_info(data=data_clf['data'], m1=data_clf['mean1'], m2=data_clf['mean2'], clf=data_clf['clf'], data_test=data_clf['data_test'], _plot=False, _print=False)
data_info['N1'] = N1
data_info['N2'] = N2
@ipywidgets.interact_manual(
    delta1=(0.9, 1., 0.01),
)
def plot(delta1=1):
    vars = {
        'delta1': delta1,
    }
    delta1 = vars['delta1']
    delta2 = deltas.delta2_given_delta1_matt(delta1, data_info)
    print(f'delta1: {delta1}, delta2: {delta2}')
    R1_est = radius.R_upper_bound(data_info['empirical R1'], data_info['R all data'], data_info['N1'], delta1)
    R2_est = radius.R_upper_bound(data_info['empirical R2'], data_info['R all data'], data_info['N2'], delta2)
    plots.plot_projection(
        data_info['projected_data'], data_info['projected_means'], R1_est, R2_est, R_est=True)

AttributeError: module 'pipeline' has no attribute 'get_data_and_classifier'

# Cost Function

In [ ]:
lims = {
    'delta1': [0, 1],
    'delta2': [0, 1],
    'c1': [0, 10],
    'c2': [0, 10],
    'N1': [1, 100],
    'N2': [1, 100],
}

@ipywidgets.interact_manual(
          plot_against=list(lims.keys()),
          delta1=(0., 1.),
          delta2=(0., 1.),
          c1=(0, 10),
          c2=(0, 10),
          N1=(1, 300),
          N2=(1, 300),
)
def plot(delta1=0.5, 
         delta2=0.5, 
         c1=1., 
         c2=1., 
         N1=100, 
         N2=100,
         plot_against='delta1'):
    vars = {
        'delta1': delta1,
        'delta2': delta2,
        'c1': c1,
        'c2': c2,
        'N1': N1,
        'N2': N2,
    }
    vars[plot_against] = np.linspace(lims[plot_against][0], lims[plot_against][1], 100)
    _, ax = plt.subplots(1, 1, figsize=(8, 6))

    J = deltas.loss(delta1=vars['delta1'], delta2=vars['delta2'], data_info=vars)

    ax.plot(vars[plot_against], J)
    ax.set_ylabel('Cost J')
    ax.set_xlabel(plot_against)
    ax.set_title(
        r"""Eq. 6: $ \text{J} = \sum_{i=1,2} C_i \left( (1-\delta_i) \frac{1}{N_i + 1}+\delta_i \right )  $""")

interactive(children=(FloatSlider(value=0.5, description='delta1', max=1.0), FloatSlider(value=0.5, descriptio…

# Cost one delta

In [ ]:
lims = {
    'delta1': [0.000000000001, 0.99999999999999999],
    'c1': [0, 10],
    'c2': [0, 10],
    'N1': [1, 100],
    'N2': [1, 100],
    'R all data':  [0.1, 3],
    'M':  [0, 2],
    'empirical R1':  [0, 1],
    'empirical R2':  [0, 1],
    'empirical D':  [1, 4],
}

@ipywidgets.interact_manual(
          plot_against=list(lims.keys()),
          delta1=(0., 1.),
          c1=(0, 10),
          c2=(0, 10),
          N1=(1, 1000),
          N2=(1, 1000),
          R=(0.1, 3.),
          M=(0., 2.),
)
def plot(delta1=0.5, 
         c1=1., 
         c2=1., 
         N1=100, 
         N2=100,
         R=1,
         M=0.5,
         R1=0.5,
         R2=0.5,
         D=3,
         plot_against='delta1'):
    vars = {
        'delta1': delta1,
        'c1': c1,
        'c2': c2,
        'N1': N1,
        'N2': N2,
        'R all data': R,
        'M': M,
        'empirical R1':  R1,
        'empirical R2':  R2,
        'empirical D':  D,
    }
    vars[plot_against] = np.linspace(lims[plot_against][0], lims[plot_against][1], 100)
    
    _, ax = plt.subplots(1, 1, figsize=(8, 6))
    vars['delta2_given_delta1_func'] = deltas.delta2_given_delta1_matt
    J = deltas.loss_one_delta(delta1=vars['delta1'], data_info=vars)
    ax.plot(vars[plot_against], J)
    ax.set_ylabel('Cost J from delta1 Matt eq 9')
    ax.set_xlabel(plot_against)

interactive(children=(FloatSlider(value=0.5, description='delta1', max=1.0), IntSlider(value=1, description='c…

# Eq. 9 - delta2 from delta1

In [ ]:
lims = {
    'delta1': [0.000000000001, 0.99999999999999999],
    'N1': [1, 100],
    'N2': [1, 100],
    'R all data': [1, 20],
    'M_emp': [1, 20],
    'empirical R1':  [0, 10],
    'empirical R2':  [0, 10],
    'empirical D':  [1, 20],
}

@ipywidgets.interact_manual(
          plot_against=list(lims.keys()),
          delta1=(0., 1.),
          N1=(1, 300),
          N2=(1, 300),
          R=(0.1, 1.),
          M_emp=(0.0, 20.),
          R1=(0.1, 0.5),
          R2=(0.1, 0.5),
          D=(0.2, 0.7),
)
def plot(delta1=0.5, 
         N1=100, 
         N2=100,
         R=1,
         M_emp=1,
         R1=0.2,
         R2=0.2,
         D=0.6,
         plot_against='delta1'):
    vars = {
        'delta1': delta1,
        'N1': N1,
        'N2': N2,
        'R all data': R,
        'M_emp': M_emp,
        'empirical R1':  R1,
        'empirical R2':  R2,
        'empirical D':  D,
    }
    vars[plot_against] = np.linspace(lims[plot_against][0], lims[plot_against][1], 100)
    
    _, ax = plt.subplots(1, 1, figsize=(8, 6))
    delta2 = deltas.delta2_given_delta1_matt(vars['delta1'], data_info=vars)
    ax.plot(vars[plot_against], delta2)
    ax.set_ylabel('delta2')
    ax.set_xlabel(plot_against)
    # ax.set_title(r"""Eq. 9: $\delta_2 = \exp \left(   -\frac{N_2}{2} \left( \frac{\hat{M}}{2R} -2\left(\sqrt{\frac{1}{N_1}+\frac{1}{N_2}}\right)- \sqrt{ \frac{ 2\ln(\frac{1}{\delta_1}) }{N_1} } \right)^2 \right) $""")
    

interactive(children=(FloatSlider(value=0.5, description='delta1', max=1.0), IntSlider(value=100, description=…

# Contraints - eq.7/8

In [ ]:
lims = {
    'delta1': [0.000000000001, 0.99999999999999999],
    'delta2': [0.000000000001, 0.99999999999999999],
    'N1': [1, 100],
    'N2': [1, 100],
    'R': [1, 20],
    'R1': [1, 20],
    'R2': [1, 20],
    'D': [1, 20],
}

@ipywidgets.interact_manual(
          delta1=(0., 1.),
          N1=(1, 300),
          N2=(1, 300),
          R=(0.1, 20.),
          R1=(0.1, 20.),
          R2=(0.1, 20.),
          D=(0.0, 20.)
)
def plot(delta1=0.5, 
         N1=100, 
         N2=100,
         R=20,
         R1=5,
         R2=5,
         D=15,):
    vars = {
        'delta1': delta1,
        'N1': N1,
        'N2': N2,
        'R': R,
        'R1': R1,
        'R2': R2,
        'D': D,
    }
    delta2_linspace = np.linspace(lims['delta2'][0], lims['delta2'][1], 100)
    
    _, ax = plt.subplots(1, 1, figsize=(8, 6))

    data_info = {
        'N1': N1,
        'N2': N2,
        'empirical R1': R1,
        'empirical R2': R2,
        'R all data': R,
        'empirical D': D,
    }
    contraint_values = deltas.contraint_eq7(vars['delta1'], delta2_linspace, data_info)
    ax.plot(delta2_linspace, contraint_values)
    ax.set_ylabel('contraint value (should equal 0)')
    ax.set_xlabel('delta2')
    ax.set_title('eq.7 against delta2')
    

interactive(children=(FloatSlider(value=0.5, description='delta1', max=1.0), IntSlider(value=100, description=…